In [130]:
# Importing Libraries to Python - added MySQLdb to remove error when connecting to Database
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql

In [131]:
# Importing launch and vehicle data and uploading it into a Data Frame
launch_data_df = pd.read_csv('launch_data_2024')

In [132]:
#Previewing CSVs
launch_data_df.head()

,launch_id,launch_date,launch_location,launch_notes
0,2023-001,03.01.2023,CC SLC-40,some satellites failed to separate
1,2023-002,08.01.2023,We LC-201,NaN
2,2023-003,09.01.2023,Jq LC-43/95B,NaN
3,2023-F01,09.01.2023,Nq RW12/30,Failed
4,2023-004,10.01.2023,CC SLC-40,NaN


In [133]:
#Database connection info
raw_host = "isba-dev-01.c5ow64oeey8b.us-east-1.rds.amazonaws.com"
raw_username = "admin"
raw_password = "isba_4715"
raw_schema = "sql_project"

# Establish a connection to the MySQL database
connection = pymysql.connect(
    host=raw_host,
    user=raw_username,
    password=raw_password,
    database=raw_schema
)
raw_engine = create_engine(f'mysql+pymysql://{raw_username}:{raw_password}@{raw_host}/{raw_schema}')

In [134]:
#importing data into mysql
table ='launch_data_2024'

launch_data_df.to_sql(name=table, con=raw_engine, if_exists="replace", index=False)

223

# Business Question: What are the top 5 locations in number of launches in 2023?

In [106]:
# Running and storing SQL query
sql_query = "WITH launch_location_ranked AS(SELECT launch_location, COUNT(*) AS number_of_launches, ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS row_num FROM launch_data_2024 GROUP BY launch_location) SELECT launch_location, number_of_launches FROM launch_location_ranked WHERE row_num <= 5;"

# Read data from MySQL into a DataFrame
df = pd.read_sql(sql_query, con=raw_engine)

df

## CC SLC-40 = America, Va SLC-4E = America, Jq LC = China, CCK = America, Xi = China

,launch_location,number_of_launches
0,CC SLC-40,55
1,Va SLC-4E,28
2,Jq LC-43/94,15
3,CCK LC-39A,13
4,Xi LC-3,9


# Insight: American launch locations makes up 3 of the top 4 locations and almost 50% of total orbital launches in 2023 (US + China = 75% of total orbintal launches in 2023)

# Recommendation: Try to sign contracts + sell to these top locations and countries (would expect places with most launches have the highest demand)

# Prediction: Increased profits

In [107]:
# Importing launch and vehicle data and uploading it into a Data Frame
vehicle_data_df = pd.read_csv('vehicle_data_2024')

In [108]:
#Previewing CSVs
vehicle_data_df.head()

,vehicle_id,vehicle_launched
0,2023-001,Falcon-9 v1.2 (Block 5)
1,2023-002,CZ-7A
2,2023-003,Ceres-1 (3) (Gushenxing-1 (3))
3,2023-F01,LauncherOne
4,2023-004,Falcon-9 v1.2 (Block 5)


In [109]:
#Adding table to database
table_2 ='vehicle_data_2024'

vehicle_data_df.to_sql(name=table_2, con=raw_engine, if_exists="replace", index=False)

223

# Business Question: The Falcon-9 and CZ vehicle families make up roughly 2/3 of launches - which launch sites use them the most?

In [125]:
#Running Query 2
sql_query_2 = "WITH common_vehicles AS (SELECT vehicle_id, vehicle_launched FROM vehicle_data_2024 vd WHERE vehicle_launched = 'Falcon-9 v1.2 (Block 5)' OR vehicle_launched LIKE 'CZ%%'), location_vehicle_raw AS (SELECT launch_location, COUNT(vehicle_launched) AS common_vehicle_launch, ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS row_num FROM common_vehicles cv JOIN launch_data_2024 ld ON cv.vehicle_id = ld.launch_id GROUP BY launch_location ORDER BY common_vehicle_launch DESC) SELECT launch_location, common_vehicle_launch FROM location_vehicle_raw WHERE row_num <= 5;"

# Try running the query with pandas
df_2 = pd.read_sql(sql_query_2, con=raw_engine)

df_2

#CC SLC-40 (55), Va SLC-4E (28), Jq LC-43/94 (15), Xi LC-3 (9), CCK LC-39A (8)


,launch_location,common_vehicle_launch
0,CC SLC-40,55
1,Va SLC-4E,28
2,Jq LC-43/94,15
3,Xi LC-3,9
4,CCK LC-39A,8


# Insights: This list is almost the exact same as the list of most launched stations (which makes sense)

# Recommendation: Get contracts selling the most common vehicles to these 3-5 launch sites

# Prediction: Increased profits